In [ ]:
#def unpairedttest(ar1,ar2):
#    npar1 = np.array(ar1)
#    npar2 = np.array(ar2)
#    return np.abs(npar1.mean()-npar2.mean())/np.sqrt((npar1.var()/npar1.shape[0])+(npar2.var()/npar2.shape[0]))

In [16]:
from model import UNet
import helper_functions as hp
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import warnings
warnings.simplefilter("ignore")
import numpy as np
import os
import scipy.io
import json
from skimage import metrics
import scipy.stats

In [17]:
def pixelwiseFPFNTP(pred,gold):
    TP = np.where(gold+pred==2,1,0).sum()
    FP = np.where(pred-gold==1,1,0).sum()
    FN = np.where(gold-pred==1,1,0).sum()

    return TP, FP, FN

def getPrecRecallFScore(TP, FP, FN):
    if (TP + FP) > 0:
        precision = TP / (TP + FP)
    else:
        precision = 0
    if (TP + FN) > 0:    
        recall = TP / (TP + FN)
    else:
        recall = 0
    if (precision+recall) > 0:    
        f_score = (2*precision*recall)/(precision+recall)
    else:
        f_score = 0
        
    return precision, recall, f_score 

def pixelwise(pred,gold):
    
    TP, FP, FN = pixelwiseFPFNTP(pred,gold)

    return getPrecRecallFScore(TP, FP, FN)

def IoU(prediction, goldB, threshold):
    goldB_cc = cv2.connectedComponents(goldB)
    pred_cc = cv2.connectedComponents(prediction)
    TP = 0
    for i in range(1,goldB_cc[0]+1):
        goldB_inst = np.where(goldB_cc[1]==i,1,0)
        
        inter_num = 0
        for j in range(1,pred_cc[0]+1):
            pred_inst = np.where(pred_cc[1]==j,1,0)
            union = goldB_inst + pred_inst
            if union.max() == 2:
                intersection = np.where(union==2,1,0)
                inter_num_curr = intersection.sum()
                if inter_num_curr > inter_num:
                    inter_num = inter_num_curr
                    union_chosen = union
                    
        if inter_num > 0:
            union_chosen = np.where(union_chosen>0,1,0)
            union_num = union_chosen.sum()
            if inter_num/union_num >= threshold:
                TP = TP + 1
                
    segmented = pred_cc[0] - 1
    positives = goldB_cc[0] - 1
    if segmented > 0:
        precision = TP / segmented
    else:
        precision = 0
    recall = TP / positives
    if precision+recall > 0:
        f_score = round(2*((precision*recall)/(precision+recall)),2)
    else:
        f_score = 0      
        
    return precision, recall, f_score

def get_weighted_hausdorff_dist(pred,gold):
    flag = False;
    if pred.sum() == 0:
        pred[0,:] = 1; pred[255,:] = 1; pred[:,0] = 1; pred[:,255] = 1;  
        flag = True
    gold_cc = cv2.connectedComponents(gold)
    pred_cc = cv2.connectedComponents(pred)
    return 0.5 * (wdh_one(pred_cc, gold_cc, pred,gold, flag) + wdh_one(gold_cc, pred_cc, gold,pred,flag))

def wdh_one(first_cc,second_cc,first,second, flag):
    whd = 0
    for i in range(1,first_cc[0]):
        first_inst = np.where(first_cc[1]==i,1,0)

        weight = first_inst.sum() / first.sum()

        if (first_inst * second).sum() > 0:
            intersection = 0
            for j in range(1,second_cc[0]):
                second_inst = np.where(second_cc[1]==j,1,0)
                intersection_new = (first_inst * second_inst).sum()
                if intersection_new > intersection:
                    hd = metrics.hausdorff_distance(first_inst==1, second_inst==1)
                    intersection = intersection_new
        else:
            if flag:
                hd = 0
                for j in range(1,second_cc[0]):
                    second_inst = np.where(second_cc[1]==j,1,0) 
                    hd_new = metrics.hausdorff_distance(first_inst==1, second_inst==1)
                    if hd_new > hd:
                        hd = hd_new                  
            else:
                hd = np.inf
                for j in range(1,second_cc[0]):
                    second_inst = np.where(second_cc[1]==j,1,0) 
                    hd_new = metrics.hausdorff_distance(first_inst==1, second_inst==1)
                    if hd_new < hd:
                        hd = hd_new  
    
        whd = whd + (weight * hd)   
    return whd        

In [18]:
first_out_channel = 32
gold_folder = 'data/goldsBinaryAll'
mn_base = ['oneTaskMaxPoolAll2/weights/allWeightsB2.pth','oneTaskMaxPoolAll2/weights/allWeightsB2_2.pth','oneTaskMaxPoolAll2/weights/allWeightsB2_3.pth','oneTaskMaxPoolAll2/weights/allWeightsB2_4.pth','oneTaskMaxPoolAll2/weights/allWeightsB2_5.pth']
mn_our = ['oneTaskMaxPoolAll_bd_25_tmp3/weights/allWeightsB2.pth','oneTaskMaxPoolAll_bd_25_tmp3/weights/allWeightsB2_2.pth','oneTaskMaxPoolAll_bd_25_tmp3/weights/allWeightsB2_3.pth','oneTaskMaxPoolAll_bd_25_tmp3/weights/allWeightsB2_4.pth','oneTaskMaxPoolAll_bd_25_tmp3/weights/allWeightsB2_5.pth']
mn_neurips = ['NeurIPSAll_25/weights/allWeightsB2.pth','NeurIPSAll_25_2/weights/allWeightsB2_2.pth','NeurIPSAll_25_3/weights/allWeightsB2_3.pth','NeurIPSAll_25_4/weights/allWeightsB2_4.pth','NeurIPSAll_25_5/weights/allWeightsB2_5.pth']
fourFolder = ['FourierNet/results_1/input-cascade_oct_L4_F16_fold2_run1/','FourierNet/results_2/input-cascade_oct_L4_F16_fold2_run1/','FourierNet/results_3/input-cascade_oct_L4_F16_fold2_run1/','FourierNet/results_4/input-cascade_oct_L4_F16_fold2_run1/','FourierNet/results_5/input-cascade_oct_L4_F16_fold2_run1/']

In [19]:
import os
import json

with open('data/vesseltypes.json') as f:
    fileTypesDict = json.load(f)
    
image_names = os.listdir('data/bucket2/ts')

typ = []

basePrecisionPixel = []
baseRecallPixel = []
baseFscorePixel = []
ourPrecisionPixel = []
ourRecallPixel = []
ourFscorePixel = []
neurPrecisionPixel = []
neurRecallPixel = []
neurFscorePixel = []
fourPrecisionPixel = []
fourRecallPixel = []
fourFscorePixel = []
basePrecisionIoU = []
baseRecallIoU = []
baseFscoreIoU = []
ourPrecisionIoU = []
ourRecallIoU = []
ourFscoreIoU = []
neurPrecisionIoU = []
neurRecallIoU = []
neurFscoreIoU = []
fourPrecisionIoU = []
fourRecallIoU = []
fourFscoreIoU = []
baseHaus = []
ourHaus = []
neurHaus = []
fourHaus = []

for i in range(5):
    for name in image_names:    
        typ.append(fileTypesDict[name])

        tst_im_name = 'data/bucket2/ts/' + name
        gold_im_name = 'data/goldsBinaryAll/' + name.split('.')[0] + '.png'

        _, _, pred_base, gold = hp.test(1, first_out_channel, mn_base[i], tst_im_name, gold_im_name)

        precision, recall, f_score = pixelwise(pred_base, gold)
        basePrecisionPixel.append(precision)
        baseRecallPixel.append(recall)
        baseFscorePixel.append(f_score)

        precision, recall, f_score = IoU(pred_base, gold, 0.5)
        basePrecisionIoU.append(precision)
        baseRecallIoU.append(recall)
        baseFscoreIoU.append(f_score)

        whd = get_weighted_hausdorff_dist(pred_base, gold)
        baseHaus.append(whd)

        _, _, pred_our, _ = hp.test(1, first_out_channel, mn_our[i], tst_im_name, gold_im_name)

        precision, recall, f_score = pixelwise(pred_our, gold)
        ourPrecisionPixel.append(precision)
        ourRecallPixel.append(recall)
        ourFscorePixel.append(f_score)

        precision, recall, f_score = IoU(pred_our, gold, 0.5)
        ourPrecisionIoU.append(precision)
        ourRecallIoU.append(recall)
        ourFscoreIoU.append(f_score)

        whd = get_weighted_hausdorff_dist(pred_our, gold)
        ourHaus.append(whd)

        _, _, pred_neurips, _ = hp.test(1, first_out_channel, mn_neurips[i], tst_im_name, gold_im_name)  

        precision, recall, f_score = pixelwise(pred_neurips, gold)
        neurPrecisionPixel.append(precision)
        neurRecallPixel.append(recall)
        neurFscorePixel.append(f_score)

        precision, recall, f_score = IoU(pred_neurips, gold, 0.5)
        neurPrecisionIoU.append(precision)
        neurRecallIoU.append(recall)
        neurFscoreIoU.append(f_score)

        whd = get_weighted_hausdorff_dist(pred_neurips, gold)
        neurHaus.append(whd)

        pred_four = cv2.imread(fourFolder[i] + name.split('.')[0] + '.png',cv2.IMREAD_GRAYSCALE)

        precision, recall, f_score = pixelwise(pred_four, gold)
        fourPrecisionPixel.append(precision)
        fourRecallPixel.append(recall)
        fourFscorePixel.append(f_score)

        precision, recall, f_score = IoU(pred_four, gold, 0.5)
        fourPrecisionIoU.append(precision)
        fourRecallIoU.append(recall)
        fourFscoreIoU.append(f_score)

        whd = get_weighted_hausdorff_dist(pred_four, gold)
        fourHaus.append(whd)   

In [20]:
def pairedttest(ar1,ar2):#we should use this because we take measurement on prediction
    arr = np.array(ar1) - np.array(ar2)
    return arr.mean()/(arr.std()/np.sqrt(arr.shape[0])),np.array(ar1).mean(),np.array(ar2).mean()

In [21]:
compare_list = [['ourPrecisionPixel',ourPrecisionPixel, basePrecisionPixel],
['ourRecallPixel',ourRecallPixel, baseRecallPixel],
['ourFscorePixel',ourFscorePixel, baseFscorePixel],
['neurPrecisionPixel',neurPrecisionPixel, basePrecisionPixel],
['neurRecallPixel',neurRecallPixel, baseRecallPixel],
['neurFscorePixel',neurFscorePixel, baseFscorePixel],
['fourPrecisionPixel',fourPrecisionPixel, basePrecisionPixel],
['fourRecallPixel',fourRecallPixel, baseRecallPixel],
['fourFscorePixel',fourFscorePixel, baseFscorePixel],
['ourPrecisionIoU',ourPrecisionIoU, basePrecisionIoU],
['ourRecallIoU',ourRecallIoU, baseRecallIoU],
['ourFscoreIoU',ourFscoreIoU, baseFscoreIoU],
['neurPrecisionIoU',neurPrecisionIoU, basePrecisionIoU],
['neurRecallIoU',neurRecallIoU, baseRecallIoU],
['neurFscoreIoU',neurFscoreIoU, baseFscoreIoU],
['fourPrecisionIoU',fourPrecisionIoU, basePrecisionIoU],
['fourRecallIoU',fourRecallIoU, baseRecallIoU],
['fourFscoreIoU',fourFscoreIoU, baseFscoreIoU],
['ourHaus',ourHaus, baseHaus],
['neurHaus',neurHaus, baseHaus],
['fourHaus',fourHaus, baseHaus],
['our-neur-PrecisionPixel',ourPrecisionPixel, neurPrecisionPixel],
['our-neur-RecallPixel',ourRecallPixel, neurRecallPixel],
['our-neur-FscorePixel',ourFscorePixel, neurFscorePixel],
['our-four-PrecisionPixel',ourPrecisionPixel, fourPrecisionPixel],
['our-four-RecallPixel',ourRecallPixel, fourRecallPixel],
['our-four-FscorePixel',ourFscorePixel, fourFscorePixel],
['our-neur-PrecisionIoU',ourPrecisionIoU, neurPrecisionIoU],
['our-neur-RecallIoU',ourRecallIoU, neurRecallIoU],
['our-neur-FscoreIoU',ourFscoreIoU, neurFscoreIoU],
['our-four-PrecisionIoU',ourPrecisionIoU, fourPrecisionIoU],
['our-four-RecallIoU',ourRecallIoU, fourRecallIoU],
['our-four-FscoreIoU',ourFscoreIoU, fourFscoreIoU],
['our-neur-Haus',ourHaus, neurHaus],
['our-four-Haus',ourHaus, fourHaus]]       

In [22]:
for item in compare_list:
    ar1 = [item[1][i] for i in range(len(item[1])) if typ[i] == 'f']
    ar2 = [item[2][i] for i in range(len(item[2])) if typ[i] == 'f']
    print(item[0], 'big')
    print(pairedttest(ar1, ar2)) #default - two sided
    print(scipy.stats.ttest_rel(ar1, ar2))
    ar1 = [item[1][i] for i in range(len(item[1])) if typ[i] != 'f']
    ar2 = [item[2][i] for i in range(len(item[2])) if typ[i] != 'f']
    print(item[0], 'other')
    print(pairedttest(ar1, ar2))
    print(scipy.stats.ttest_rel(ar1, ar2))
    print(item[0], 'all')
    print(pairedttest(item[1], item[2]))
    print(scipy.stats.ttest_rel(item[1], item[2]))    

ourPrecisionPixel big
(6.541067960864502, 0.9351921150281719, 0.930609372237531)
Ttest_relResult(statistic=6.540115771040568, pvalue=7.058337493267227e-11)
ourPrecisionPixel other
(3.936982205690797, 0.8319949695457526, 0.8247625986387436)
Ttest_relResult(statistic=3.9364530058489424, pvalue=8.420787401355987e-05)
ourPrecisionPixel all
(5.885467366826605, 0.8815382532259917, 0.8755779260058764)
Ttest_relResult(statistic=5.885056068921437, pvalue=4.158886609941077e-09)
ourRecallPixel big
(4.308166437928626, 0.9451716226996623, 0.9413105075637006)
Ttest_relResult(statistic=4.307539293819047, pvalue=1.6970358091677647e-05)
ourRecallPixel other
(10.18806147830402, 0.8052212395274849, 0.7839714603146689)
Ttest_relResult(statistic=10.186692023162635, pvalue=4.6915440883457126e-24)
ourRecallPixel all
(11.016141349151415, 0.8724091593313185, 0.8595073970442879)
Ttest_relResult(statistic=11.015371501053117, pvalue=5.404064825211508e-28)
ourFscorePixel big
(6.015865786715492, 0.9372948291603939,

our-neur-FscoreIoU other
(0.556999714183737, 0.7646102150537634, 0.7631451612903226)
Ttest_relResult(statistic=0.5569248435987918, pvalue=0.5776122922105862)
our-neur-FscoreIoU all
(5.735706065761578, 0.830566037735849, 0.8189671558350805)
Ttest_relResult(statistic=5.7353052337218315, pvalue=1.0130533854089096e-08)
our-four-PrecisionIoU big
(-7.793139669654942, 0.8697218388921445, 0.9006889859291607)
Ttest_relResult(statistic=-7.792005214496496, pvalue=8.673922256565683e-15)
our-four-PrecisionIoU other
(5.080258739008128, 0.7635084248189087, 0.74018956849602)
Ttest_relResult(statistic=5.079575862637141, pvalue=3.968600257771092e-07)
our-four-PrecisionIoU all
(-0.8928927642496188, 0.814499770359309, 0.8172427479345717)
Ttest_relResult(statistic=-0.892830365649714, pvalue=0.3719780533558952)
our-four-RecallIoU big
(5.842254030272422, 0.9673944687045124, 0.9502183406113537)
Ttest_relResult(statistic=5.841403567493294, pvalue=5.6586104361177755e-09)
our-four-RecallIoU other
(31.88905651478

In [23]:
def IoU_accm(prediction, goldB, threshold):
    goldB_cc = cv2.connectedComponents(goldB)
    pred_cc = cv2.connectedComponents(prediction)
    TP = 0
    for i in range(1,goldB_cc[0]+1):
        goldB_inst = np.where(goldB_cc[1]==i,1,0)
        
        inter_num = 0
        for j in range(1,pred_cc[0]+1):
            pred_inst = np.where(pred_cc[1]==j,1,0)
            union = goldB_inst + pred_inst
            if union.max() == 2:
                intersection = np.where(union==2,1,0)
                inter_num_curr = intersection.sum()
                if inter_num_curr > inter_num:
                    inter_num = inter_num_curr
                    union_chosen = union
                    
        if inter_num > 0:
            union_chosen = np.where(union_chosen>0,1,0)
            union_num = union_chosen.sum()
            if inter_num/union_num >= threshold:
                TP = TP + 1
                
    segmented = pred_cc[0] - 1
    positives = goldB_cc[0] - 1     
        
    return segmented, positives, TP

def getPrecRecallFScore_IoU_accm(segmented,positives,TP):
    precision = TP/segmented
    recall = TP/positives
    f_score = 2*((precision*recall)/(precision+recall))
    return precision, recall, f_score

def getResultsIoU_accm(patient, model_name):
    
    with open('data/vesseltypes.json') as f:
        fileTypesDict = json.load(f)
            
    image_names = [im_name for im_name in os.listdir('data/bucket2/ts') if im_name.split('_')[0] == patient]
        
    segmented_sum_f = 0
    positives_sum_f = 0
    TP_sum_f = 0
    segmented_sum_s = 0
    positives_sum_s = 0
    TP_sum_s = 0
                
    for name in image_names:
        tst_im_name = 'data/bucket2/ts/' + name
        gold_im_name = 'data/goldsBinaryAll/' + name.split('.')[0] + '.png'
        
        if model_name[:7] == 'Fourier':
            prediction = cv2.imread(model_name + name.split('.')[0] + '.png',cv2.IMREAD_GRAYSCALE)
            goldB = cv2.imread(gold_im_name,cv2.IMREAD_GRAYSCALE)
        else:    
            _, _, prediction, goldB = hp.test(1, 32, model_name, tst_im_name, gold_im_name)

        segmented, positives, TP = IoU_accm(prediction, goldB, 0.5)

        if fileTypesDict[name] == 'f':
            segmented_sum_f = segmented_sum_f + segmented
            positives_sum_f = positives_sum_f + positives
            TP_sum_f = TP_sum_f + TP
        else:
            segmented_sum_s = segmented_sum_s + segmented
            positives_sum_s = positives_sum_s + positives
            TP_sum_s = TP_sum_s + TP
    
    segmented_sum = segmented_sum_f + segmented_sum_s
    positives_sum = positives_sum_f + positives_sum_s
    TP_sum = TP_sum_f + TP_sum_s
    results = [0,0,0,0,0,0,0,0,0]
    
    precision, recall, f_score = getPrecRecallFScore_IoU_accm(segmented_sum,positives_sum,TP_sum)    
    results[0] = round(precision,4)
    results[1] = round(recall,4)
    results[2] = round(f_score,4)
    
    precision, recall, f_score = getPrecRecallFScore_IoU_accm(segmented_sum_f,positives_sum_f,TP_sum_f)    
    results[3] = round(precision,4)
    results[4] = round(recall,4)
    results[5] = round(f_score,4)
    
    precision, recall, f_score = getPrecRecallFScore_IoU_accm(segmented_sum_s,positives_sum_s,TP_sum_s)    
    results[6] = round(precision,4)
    results[7] = round(recall,4)
    results[8] = round(f_score,4)  
    
    return results

In [24]:
patients = np.unique([p_im.split('_')[0] for p_im in os.listdir('data/bucket2/ts')])

In [25]:
base = []
our = []
neur = []
four = []

In [26]:
models = [[base,mn_base], [our,mn_our], [neur,mn_neurips], [four,fourFolder]]

In [27]:
for model in models:
    for patient in patients:
        for i in range(5): 
            model[0].append(getResultsIoU_accm(patient, model[1][i]))

In [28]:
headers = ['precision all','recall all','f-score all','precision big','recall big','f-score big','precision others','recall others','f-score others']

In [29]:
base = np.array(base)
our = np.array(our)
neur = np.array(neur)
four = np.array(four)

In [30]:
print('our-base:')
for i in range(len(headers)):
    print(headers[i])
    print(scipy.stats.ttest_rel(our[:,i], base[:,i]))
print('our-neur:')
for i in range(len(headers)):
    print(headers[i])
    print(scipy.stats.ttest_rel(our[:,i], neur[:,i]))    
print('our-four:')
for i in range(len(headers)):
    print(headers[i])
    print(scipy.stats.ttest_rel(our[:,i], four[:,i]))     

our-base:
precision all
Ttest_relResult(statistic=3.0811090308015396, pvalue=0.0037720927040101973)
recall all
Ttest_relResult(statistic=1.9948820067130948, pvalue=0.05307744028662136)
f-score all
Ttest_relResult(statistic=2.870946081972609, pvalue=0.006581735638382635)
precision big
Ttest_relResult(statistic=2.7663207673778607, pvalue=0.008620584451931666)
recall big
Ttest_relResult(statistic=1.33940444178711, pvalue=0.1881928147443136)
f-score big
Ttest_relResult(statistic=2.6636650128892843, pvalue=0.0111779963620057)
precision others
Ttest_relResult(statistic=2.47720707800007, pvalue=0.017681845885845093)
recall others
Ttest_relResult(statistic=1.639142717527566, pvalue=0.10922686881577585)
f-score others
Ttest_relResult(statistic=2.3072872263117707, pvalue=0.026434578757743382)
our-neur:
precision all
Ttest_relResult(statistic=2.3042076928158473, pvalue=0.026624088119336534)
recall all
Ttest_relResult(statistic=-0.25480813978171046, pvalue=0.8002101760795204)
f-score all
Ttest_rel